# RF wrapper using Mlxtend is in the 2nd section

## Section 1: Performing KS and FDR

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sfs ####MLXTEND LIBRARY
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [1]:
model = pd.read_csv('FinalCleanedVariablesNoNa.csv') 
# file that after features creation and NA values were removed, and ready for feature selection

In [2]:
date = pd.read_csv("ApplicationsFrivLRemo.csv") 
# the file that after data cleaning and processing, and ready for variable creation 

In [3]:
date.head()

,Unnamed: 0,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,0,1,20160101,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,2765,1,1797504115,0
1,1,2,20160101,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,2,3,20160101,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,3,216537580,0
3,3,4,20160101,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0
4,4,5,20160101,24065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,3173,19980315,6101082272,0


In [4]:
date_new = date.loc[date['date'] < 20161101,:] # get traning+test set

In [5]:
date_new.tail()
#This function returns last n rows from the object based on position. 
#It is useful for quickly verifying data, for example, after sorting or appending rows.

,Unnamed: 0,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
833502,833502,833503,20161031,614489026,EJUJRSUMZ,EEJSTZUT,9034 UERXU RD,4529,20130609,3743139222,0
833503,833503,833504,20161031,20529638,RZRTEJXUT,UXMSSZSX,1452 RJMMA ST,68562,19550212,7155993451,0
833504,833504,833505,20161031,604655517,UTEXRMERT,EZRXRXEZ,8381 RAXMU ST,52826,19470127,833505,0
833505,833505,833506,20161031,171974215,EXMUXAZRX,SZUTAURS,9517 XXMMZ ST,58097,19290208,9747929283,0
833506,833506,833507,20161031,985717679,RXMJSAJZE,REAJMAEZ,4205 EZJJU BLVD,9416,19250709,5386413150,0


In [6]:
model = model.iloc[:,1:]

In [7]:
date['fraud_label'].sum() #get how many fraud label==1

14393

In [8]:
model1 = model.rename(columns = {'fraud_label': 'FraudLabel'})

In [9]:
model1.head()

,record,diff_date.ssn,diff_date.fulladdress,diff_date.nameDOB,diff_date.homephone,diff_date.ssn_fulladdress,diff_date.ssn_firstname,diff_date.ssn_lastname,diff_date.ssn_nameDOB,diff_date.ssn_homephone,...,fulladdresshomephone7,fulladdresshomephone14,fulladdresshomephone30,fulladdressnameDOB0,fulladdressnameDOB1,fulladdressnameDOB3,fulladdressnameDOB7,fulladdressnameDOB14,fulladdressnameDOB30,FraudLabel
0,1,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,...,1,1,1,1,1,1,1,1,1,0
1,2,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,...,1,1,1,1,1,1,1,1,1,1
2,3,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,...,1,1,1,1,1,1,1,1,1,0
3,4,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,...,1,1,1,1,1,1,1,1,1,0
4,5,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,365.0,...,1,1,1,1,1,1,1,1,1,0


In [10]:
model['fraud_label'].sum()

14393

In [11]:
modelTrainTest = model.loc[(model['record'] <= 833507),:] #get train+test set, #833507 is for 10.31
modelTrainTest.head(5)
modelTrainTest.shape

(833507, 103)

In [ ]:
KS_FDR = pd.read_excel('HW11_new.xlsx') #reading the file containing feature importance after Filter (KS, FDR) step

In [14]:
KS_FDR.head(5)

,Unnamed: 0,Variable,KS,FDR
0,0,FraudLabel,1.000000,1.0
1,1,risk_fulladdressnameDOB,0.996150,1.0
2,2,risk_nameDOBhomephone,0.996046,1.0
3,3,risk_ssnfulladdress,0.995949,1.0
4,4,risk_ssnhomephone,0.995626,1.0


In [141]:
#ksfdr1 = KS_FDR.drop(columns = ['Avg (both)'])
#ksfdr1.head()

In [15]:
ksfdr2 = KS_FDR.sort_values(by = 'KS', ascending = False)
ksfdr2.head()

,Unnamed: 0,Variable,KS,FDR
0,0,FraudLabel,1.000000,1.0
1,1,risk_fulladdressnameDOB,0.996150,1.0
2,2,risk_nameDOBhomephone,0.996046,1.0
3,3,risk_ssnfulladdress,0.995949,1.0
4,4,risk_ssnhomephone,0.995626,1.0


In [16]:
for i in range(len(ksfdr2)):   #rank order for KS in ascending order
    ksfdr2.iloc[i,2] = i+1
ksfdr2.head(5)

,Unnamed: 0,Variable,KS,FDR
0,0,FraudLabel,1.0,1.0
1,1,risk_fulladdressnameDOB,2.0,1.0
2,2,risk_nameDOBhomephone,3.0,1.0
3,3,risk_ssnfulladdress,4.0,1.0
4,4,risk_ssnhomephone,5.0,1.0


In [17]:
ksfdr2.tail(5)

,Unnamed: 0,Variable,KS,FDR
98,98,ssnhomephone0,99.0,0.041559
99,99,ssnfulladdress0,100.0,0.041559
100,100,dtnameDOB14,101.0,0.040060
101,101,ssn14,102.0,0.040060
102,102,fulladdress14,103.0,0.040060


In [18]:
ksfdr3 = ksfdr2.sort_values(by = 'FDR', ascending = False)
ksfdr3.head()

,Unnamed: 0,Variable,KS,FDR
0,0,FraudLabel,1.0,1.0
7,7,risk_nameDOB,8.0,1.0
1,1,risk_fulladdressnameDOB,2.0,1.0
11,11,risk_ssn,12.0,1.0
10,10,risk_fulladdress,11.0,1.0


In [19]:
for i in range(len(ksfdr3)):   #rank order for score FDR in ascending order
    ksfdr3.iloc[i,3] = i+1
ksfdr3.head(5)

,Unnamed: 0,Variable,KS,FDR
0,0,FraudLabel,1.0,1.0
7,7,risk_nameDOB,8.0,2.0
1,1,risk_fulladdressnameDOB,2.0,3.0
11,11,risk_ssn,12.0,4.0
10,10,risk_fulladdress,11.0,5.0


In [20]:
ksfdr3['Cumulative score'] = (ksfdr3['KS'] + ksfdr3['FDR'])/2
ksfdr3.head()

,Unnamed: 0,Variable,KS,FDR,Cumulative score
0,0,FraudLabel,1.0,1.0,1.0
7,7,risk_nameDOB,8.0,2.0,5.0
1,1,risk_fulladdressnameDOB,2.0,3.0,2.5
11,11,risk_ssn,12.0,4.0,8.0
10,10,risk_fulladdress,11.0,5.0,8.0


In [21]:
ksfdr4 = ksfdr3.sort_values(by = 'Cumulative score', ascending = True)
ksfdr4.head(20)

,Unnamed: 0,Variable,KS,FDR,Cumulative score
0,0,FraudLabel,1.0,1.0,1.0
1,1,risk_fulladdressnameDOB,2.0,3.0,2.5
7,7,risk_nameDOB,8.0,2.0,5.0
3,3,risk_ssnfulladdress,4.0,11.0,7.5
4,4,risk_ssnhomephone,5.0,10.0,7.5
5,5,risk_fulladdresshomephone,6.0,9.0,7.5
6,6,risk_ssnnameDOB,7.0,8.0,7.5
2,2,risk_nameDOBhomephone,3.0,12.0,7.5
8,8,risk_ssnfirstname,9.0,6.0,7.5
10,10,risk_fulladdress,11.0,5.0,8.0


In [22]:
ksfdr4.shape

(103, 5)

In [24]:
c = list(ksfdr4.iloc[:70,1]) #dropping half of the variables with high KS, FDR ranks (lower in table)

In [25]:
len(c)

70

In [26]:
modelTrainTest.isnull().sum().sum() #making sure there are no null values anyhwere

0

In [27]:
modelTrainTest = modelTrainTest.rename(columns = {'fraud_label': 'FraudLabel'})

In [28]:
modelv = modelTrainTest.loc[:,c] #selecting only the top columns after KS and FDR
modelv.head() 

,FraudLabel,risk_fulladdressnameDOB,risk_nameDOB,risk_ssnfulladdress,risk_ssnhomephone,risk_fulladdresshomephone,risk_ssnnameDOB,risk_nameDOBhomephone,risk_ssnfirstname,risk_fulladdress,...,fulladdressnameDOB30,nameDOBhomephone30,diff_date.ssn_lastname,ssnhomephone30,diff_date.fulladdress_homephone,ssnfulladdress30,ssnhomephone14,fulladdressnameDOB14,diff_date.lastname_nameDOB,nameDOBhomephone14
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1,1,365.0,1,365.0,1,1,1,365.0,1
1,1,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,...,1,1,365.0,1,365.0,1,1,1,365.0,1
2,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1,1,365.0,1,365.0,1,1,1,365.0,1
3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1,1,365.0,1,365.0,1,1,1,365.0,1
4,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1,1,365.0,1,365.0,1,1,1,365.0,1


In [32]:
modelvv = modelv.drop(['risk_fulladdresshomephone','risk_ssnnameDOB','risk_nameDOBhomephone','risk_ssnfirstname','risk_fulladdress',
             'risk_ssn','risk_ssnlastname','risk_homephone'],axis = 1)

In [33]:
modelvv.columns

Index(['FraudLabel', 'risk_fulladdressnameDOB', 'risk_nameDOB',
       'risk_ssnfulladdress', 'risk_ssnhomephone', 'fulladdress30',
       'fulladdress7', 'fulladdress3', 'fulladdress1', 'dtnameDOB30', 'ssn30',
       'ssnfirstname30', 'ssnnameDOB30', 'ssnlastname30',
       'fulladdresshomephone30', 'fulladdresshomephone14', 'ssnfirstname14',
       'ssnnameDOB14', 'ssnlastname14', 'fulladdresshomephone7', 'dtnameDOB7',
       'ssn7', 'ssnfirstname7', 'homephone3', 'ssnnameDOB7', 'ssnlastname7',
       'fulladdress0', 'fulladdresshomephone3', 'dtnameDOB3', 'homephone1',
       'ssn3', 'ssnfirstname3', 'ssnlastname3', 'ssnnameDOB3',
       'fulladdresshomephone1', 'dtnameDOB1', 'ssn1', 'ssnfirstname1',
       'ssnlastname1', 'ssnnameDOB1', 'homephone0', 'fulladdresshomephone0',
       'diff_date.fulladdress', 'ssn0', 'ssnfirstname0', 'ssnlastname0',
       'dtnameDOB0', 'diff_date.ssn', 'diff_date.nameDOB', 'ssnnameDOB0',
       'diff_date.ssn_firstname', 'diff_date.ssn_nameDOB',
     

In [128]:
modelvv.to_csv("FinalFilteredVariables.csv")

In [34]:
ytrain = modelvv.loc[:,'FraudLabel'] #splitting the table into a column containing the label variable

In [35]:
ytrain.head()

0    0
1    1
2    0
3    0
4    0
Name: FraudLabel, dtype: int64

In [150]:
ytrain.shape

(833507,)

In [36]:
xtrain = modelvv.drop(columns ='FraudLabel') #removing Fraud from the table containing predictor variables

In [2]:
#xtrain.shape

## Feature Selection using RF step forward - Mlxtend library

In [161]:
%%time
np.random.seed(1)
# Build RF classifier to use in feature selection
clf = RandomForestClassifier(n_estimators=10, n_jobs=-1)

# Build step forward feature selection
sfs1 = sfs(clf, k_features=20, forward=True, floating=False, verbose=2, scoring='accuracy', cv=3)

# Perform SFFS
sfs1 = sfs1.fit(xtrain, ytrain)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  61 out of  61 | elapsed:  7.0min finished

[2019-04-27 23:02:47] Features: 1/20 -- score: 0.9977696647787994[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  5.8min finished

[2019-04-27 23:08:37] Features: 2/20 -- score: 0.9987102690793659[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  59 out of  59 | elapsed:  6.3min finished

[2019-04-27 23:14:57] Features: 3/20 -- score: 0.9990018081757724[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

Wall time: 2h 35min 27s


[Parallel(n_jobs=1)]: Done  42 out of  42 | elapsed:  9.0min finished

[2019-04-28 01:31:14] Features: 20/20 -- score: 0.9990737932000088

In [309]:
feat_cols20 = list(sfs1.k_feature_idx_) ##Indetifying the indexes of 20 variables

In [304]:
feat_cols20

[0, 1, 2, 3, 4, 11, 12, 14, 16, 18, 19, 21, 24, 25, 30, 39, 42, 45, 51, 58]

In [310]:
del feat_cols20[0:4]  #since risk variables led to overfitting

In [311]:
feat_cols20

[4, 11, 12, 14, 16, 18, 19, 21, 24, 25, 30, 39, 42, 45, 51, 58]

In [42]:
redVar = xtrain.iloc[:,feat_cols20]
VarName = list(redVar.columns)
VarName

['fulladdress30',
 'dtnameDOB30',
 'ssn30',
 'ssnnameDOB30',
 'ssnlastname30',
 'fulladdresshomephone30',
 'fulladdresshomephone14',
 'dtnameDOB7',
 'fulladdress0',
 'ssnfirstname3',
 'ssnlastname3',
 'ssnnameDOB3',
 'fulladdresshomephone1',
 'dtnameDOB1',
 'homephone0',
 'ssn0',
 'ssnfirstname0',
 'ssnlastname0',
 'dtnameDOB0',
 'ssnnameDOB0',
 'fulladdressnameDOB30']

In [43]:
#With the output label
VarName1 = VarName + ['FraudLabel']